In [ ]:
#default_exp data.declarative
%load_ext autoreload
%autoreload 2

In [ ]:
# export
# hide
import json
import re
import os
import requests
import urllib

from pymemri.data.itembase import Item

In [ ]:
# export
# hide

def schema_init(self, **kwargs):
    super(type(self), self).__init__(**kwargs)

    for p in self._property_kwargs:
        setattr(self, p, kwargs.get(p, None))
        
    for e in self._edge_kwargs:
        setattr(self, e, kwargs.get(e, list()))
    
class SchemaFactory:
    
    @staticmethod
    def is_valid_schema_name(name: str) -> bool:
        """Check if name is a valid schema name"""
        return re.match(r"[A-Z][a-zA-Z0-9_]*", name)

    @staticmethod
    def create_schema(name: str, properties: list, edges: list, 
                      base: type=Item, description=None) -> type:
        """
        Create a type for schema from a name, list of properties and list of edges. Optionally set the base schema.
        """
        if not  SchemaFactory.is_valid_schema_name(name):
            raise ValueError(f"{name} is not a valid schema name.")

        properties = [p for p in properties if p not in base.properties]
        edges = [e for e in edges if e not in base.edges]
        
        # Store property_kwargs and edge_kwargs of self to set these in the init.
        cls_dict = {
            "_property_kwargs": properties, 
            "_edge_kwargs": edges,
            "properties": base.properties + properties,
            "edges": base.edges + edges,
            "description": description,
            "__init__": schema_init
        }
        
        schema_cls = type(
            name, 
            (base,), 
            cls_dict
        )
        return schema_cls
    
    @staticmethod
    def from_dict(name: str, schema_dict: dict) -> type:
        """
        Creates a schema type from a dictionary. See https://gitlab.memri.io/memri/schema/ for format.
        
        TODO add edge constraints ("sequenced" and "singular")
        TODO add option for different base class
        """
        properties = schema_dict.get("properties", [])
        edges = list(schema_dict.get("relations", {}).keys())
        description=schema_dict.get("description", None)
        
        return SchemaFactory.create_schema(
            name=name,
            properties=properties,
            edges=edges,
            description=description
        )
        
    @staticmethod
    def from_file(file_name: str) -> type:
        """
        Create a schema class from json file.
        """
        schema_name = os.path.splitext(os.path.basename(file_name))[0]
        schema_dict = json.load(file_name)
        return SchemaFactory.from_dict(schema_name, schema_dict)
    
    @staticmethod
    def from_url(url: str) -> type:
        """
        Create a schema class from json url.
        """
        file_name = urllib.parse.urlparse(url)[2].rpartition('/')[-1]
        schema_name = os.path.splitext(file_name)[0]
        schema_dict = requests.get(url).json()
        return SchemaFactory.from_dict(schema_name, schema_dict)

In [ ]:
from pymemri.pod.client import PodClient
pod_client = PodClient()

In [ ]:
account_json = """
{
  "description": "An account or subscription, for instance for some online service, or a bank account or wallet.",
  "properties": [
    "handle",
    "displayName",
    "service",
    "itemType",
    "avatarUrl"
  ],
  "relations": {
    "belongsTo": {
      "sequenced": false,
      "singular": false
    },
    "price": {
      "sequenced": false,
      "singular": false
    },
    "location": {
      "sequenced": false,
      "singular": false
    },
    "organization": {
      "sequenced": false,
      "singular": false
    },
    "contact": {
      "sequenced": false,
      "singular": false
    }
  },
  "foregroundColor": "#ffffff",
  "backgroundColor": "#93c47d"
}
""" 

Account = SchemaFactory.from_dict("Account", json.loads(account_json))
test_account = Account(handle="Test Account", displayName="Test Account")

assert pod_client.add_to_schema(Account(handle="", displayName=""))
assert pod_client.create(test_account)

In [ ]:
url = 'https://gitlab.memri.io/memri/schema/-/raw/dev/TypeHierarchy/Item/Account/Account.json'
Account = SchemaFactory.from_url(url)
test_account = Account(handle="Test Account", displayName="Test Account")

assert pod_client.add_to_schema(Account(handle="", displayName=""))
assert pod_client.create(test_account)

In [ ]:
print(pod_client.search({"type": "Account"}))

[Account (#779f42c8cd3a246c277ef65f8c0fc795), Account (#36873234c80964b000fd23a6e33b45f0)]


In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted basic.ipynb.
Converted data.declarative.ipynb.
Converted data.photo.ipynb.
Converted importers.Importer.ipynb.
Converted importers.util.ipynb.
Converted index.ipynb.
Converted indexers.indexer.ipynb.
Converted itembase.ipynb.
Converted plugin.pluginbase.ipynb.
Converted plugin.schema.ipynb.
Converted plugin.stateful.ipynb.
Converted pod.client.ipynb.
Converted pod.db.ipynb.
Converted pod.utils.ipynb.
